In [39]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import gc
import time
import sys
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

from catboost import Pool, CatBoostClassifier



In [6]:
#CatBoost

train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

idx = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

df_list = [train_df, test_df]
new_df_list = []
new_train_df = pd.DataFrame()
new_train_df['target'] = train_df['target']
new_train_df['ID_code'] = train_df['ID_code']
new_test_df = pd.DataFrame()
new_test_df['ID_code'] = test_df['ID_code']
new_df_list = [new_train_df, new_test_df]
for i in range(0, len(df_list)):        
    for feat in idx:
        mean = df_list[i].loc[:,feat].mean()
        new_df_list[i][feat] = df_list[i][feat]
#         new_df_list[i]["mm_" + feat] = np.round(df_list[i][feat] - mean , 2) 
#         if mean > 0:
#             new_df_list[i]["is_more_" + feat] = np.where( df_list[i][feat] > mean, 10, -10 )
#         if mean < 0: 
#             new_df_list[i]["is_more_" + feat] = np.where( df_list[i][feat] < mean, 10, -10 )
            
features = [c for c in new_train_df.columns if c not in ['ID_code', 'target']]
target = new_train_df['target']

N_FOLDS = 5.0

model = CatBoostClassifier(loss_function="Logloss", eval_metric="AUC", iterations=2000, early_stopping_rounds= 500, l2_leaf_reg= 9, learning_rate= 0.1)
#model = CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 1500, l2_leaf_reg= 9, learning_rate= 0.15)


kf = KFold(n_splits=int(N_FOLDS), random_state=2019, shuffle=True)

y_valid_pred = 0 * target
y_test_pred = 0
y_train_pred = 0 #storing results for ensemble

new_train_df.info()

for idx, (train_index, valid_index) in enumerate(kf.split(new_train_df)):
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_train, X_valid = new_train_df[features].iloc[train_index,:], new_train_df[features].iloc[valid_index,:]
    _train = Pool(X_train, label=y_train)
    _valid = Pool(X_valid, label=y_valid)
    print( "\nFold ", idx)
    fit_model = model.fit(_train,
                          eval_set=_valid,
                          use_best_model=True,
                          verbose=200
                         )
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    y_valid_pred.iloc[valid_index] = pred
    y_test_pred += fit_model.predict_proba(new_test_df[features])[:,1]
    y_train_pred += fit_model.predict_proba(new_train_df[features])[:,1]
y_test_pred /= N_FOLDS
y_train_pred /= N_FOLDS

#save an intermediate submission
sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred
sub_df1.to_csv("submission_catboost.csv", index=False)

ens_cat_df = pd.DataFrame({"ID_code":train_df["ID_code"].values})
ens_cat_df["output"] = y_train_pred
ens_cat_df.to_csv("../input/catboost_output.csv")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, target to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB

Fold  0
0:	test: 0.5364710	best: 0.5364710 (0)	total: 130ms	remaining: 4m 20s
200:	test: 0.8776868	best: 0.8776868 (200)	total: 22.6s	remaining: 3m 21s
400:	test: 0.8900528	best: 0.8900528 (400)	total: 45.2s	remaining: 3m
600:	test: 0.8947133	best: 0.8947133 (600)	total: 1m 7s	remaining: 2m 36s
800:	test: 0.8963380	best: 0.8963380 (800)	total: 1m 29s	remaining: 2m 13s
1000:	test: 0.8966581	best: 0.8967353 (993)	total: 1m 51s	remaining: 1m 51s
1200:	test: 0.8970510	best: 0.8970800 (1194)	total: 2m 13s	remaining: 1m 29s
1400:	test: 0.8970907	best: 0.8972818 (1294)	total: 2m 35s	remaining: 1m 6s
1600:	test: 0.8970987	best: 0.8972818 (1294)	total: 2m 57s	remaining: 44.2s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.8972817673
bestIteration = 1294

Shrink model to first 1295 iterations

In [40]:
#####LightGBM
param = {
            'num_leaves': 15, #was 10
            'max_bin': 119,
            'min_data_in_leaf': 11,
            'learning_rate': 0.01,
            'min_sum_hessian_in_leaf': 0.00245,
            'bagging_fraction': 1.0, 
            'bagging_freq': 5, 
            'feature_fraction': 0.05,
            'lambda_l1': 4.972,
            'lambda_l2': 50.0,   #30 #2.276,
            'min_gain_to_split': 0.65,
            'max_depth': 20, #17 #was 14
            'save_binary': True,
            'seed': 1337,
            'feature_fraction_seed': 1337,
            'bagging_seed': 1337,
            'drop_seed': 1337,
            'data_random_seed': 1337,
            'objective': 'binary',
            'boosting_type': 'gbdt',
            'verbose': 1,
            'metric': 'auc',
            'is_unbalance': True,
            'boost_from_average': True, #was false
        }
num_round = 15000
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
train_preds = np.zeros(len(train_df))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
    train_preds += clf.predict(train_df[features], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub_lgb = pd.DataFrame({"ID_code": train_df.ID_code.values})
sub["target"] = predictions
sub_lgb["output"] = train_preds
sub.to_csv("submission_lgbm.csv", index=False)
sub_lgb.to_csv("../input/lgbm_output.csv")

Fold 0
Training until validation scores don't improve for 200 rounds.
[1000]	training's auc: 0.905752	valid_1's auc: 0.885235
[2000]	training's auc: 0.916553	valid_1's auc: 0.891313
[3000]	training's auc: 0.925128	valid_1's auc: 0.8953
[4000]	training's auc: 0.931935	valid_1's auc: 0.897273
[5000]	training's auc: 0.937554	valid_1's auc: 0.898575
[6000]	training's auc: 0.942476	valid_1's auc: 0.89927
[7000]	training's auc: 0.946842	valid_1's auc: 0.899761
Early stopping, best iteration is:
[6992]	training's auc: 0.946805	valid_1's auc: 0.899777


KeyboardInterrupt: 

In [36]:
##NN Ensemble
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(2,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

print(model.summary())

N_FOLDS = 5.0

kf = KFold(n_splits=int(N_FOLDS), random_state=2019, shuffle=True)

#Load previous data



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 64)                192       
_________________________________________________________________
dense_11 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_12 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 65        
Total params: 8,577
Trainable params: 8,577
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
catb_df = pd.read_csv("../input/catboost_output.csv")
lgbm_df = pd.read_csv("../input/lgbm_output.csv")

submission_catb_df = pd.read_csv("submission_catboost.csv")
submission_lgbm_df = pd.read_csv("submission_lgbm.csv")

combined_df = pd.DataFrame({"ID_code": catb_df.ID_code.values})
combined_df['cat_out'] = catb_df['output']
combined_df['lgbm_out'] = lgbm_df['output']
combined_df['target'] = train_df['target']

combined_test_df = pd.DataFrame({"ID_code": test_df.ID_code.values})
combined_test_df['cat_out'] = submission_catb_df['target']
combined_test_df['lgbm_out'] = submission_lgbm_df['target']


combined_df.to_csv('../input/combined.csv')

#idx = [c for c in combined_df.columns if c not in ['ID_code', 'target']]
features = [c for c in combined_df.columns if c not in ['ID_code', 'target']]
target = combined_df['target']

y_test_pred = np.zeros(len(test_df))
    
for idx, (train_index, valid_index) in enumerate(kf.split(combined_df)):
    print("Fold {}".format(idx))
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_train, X_valid = combined_df[features].iloc[train_index,:], combined_df[features].iloc[valid_index,:]
    model.fit(X_train, y_train, epochs=5, batch_size = 512, validation_data =(X_valid, y_valid))
    pred = model.predict_proba(X_valid)
    print(pred.shape)
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    y_test_pred += model.predict_proba(combined_test_df[features])[:,0]
    print(y_test_pred.shape)

sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred / N_FOLDS
sub_df1.to_csv("submission_nn.csv", index=False)


Fold 0
Train on 160000 samples, validate on 40000 samples
Epoch 1/5
160000/160000 [==============================] - 2s 12us/step - loss: 0.1933 - acc: 0.9388 - val_loss: 0.1452 - val_acc: 0.9475
Epoch 2/5
160000/160000 [==============================] - 2s 9us/step - loss: 0.1427 - acc: 0.9478 - val_loss: 0.1464 - val_acc: 0.9456
Epoch 3/5
160000/160000 [==============================] - 1s 9us/step - loss: 0.1397 - acc: 0.9492 - val_loss: 0.1345 - val_acc: 0.9520
Epoch 4/5
160000/160000 [==============================] - 1s 9us/step - loss: 0.1364 - acc: 0.9519 - val_loss: 0.1302 - val_acc: 0.9544
Epoch 5/5
160000/160000 [==============================] - 1s 9us/step - loss: 0.1332 - acc: 0.9544 - val_loss: 0.1266 - val_acc: 0.9587
(40000, 1)
  auc =  0.9586908932205548
(200000,)
Fold 1
Train on 160000 samples, validate on 40000 samples
Epoch 1/5
160000/160000 [==============================] - 1s 9us/step - loss: 0.1312 - acc: 0.9563 - val_loss: 0.1217 - val_acc: 0.9603
Epoch 2/5
16